In [ ]:
# Colab bootstrap: install dependencies
!pip install pandas numpy matplotlib statsmodels arch fredapi pandas-datareader tqdm


## 1  Setup & Data Load

In this first section we:

1. Import required libraries.  
2. Pull our key series (Real GDP, Potential GDP, CPI, IP, Fed Funds).  
3. Provide a quick preview of the data.  
4. Define helper routines for subsequent plots and tests.


In [ ]:
# 1.2 — Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader import data as web

# 1.3 — Date range
start = "1960-01-01"
end   = pd.Timestamp.today().strftime("%Y-%m-%d")

# 1.4 — FRED tickers dictionary
tickers = {
    "GDP":    "GDPC1",      # Real GDP (qtrly, SA)
    "POT_GDP":"GDPPOT",     # Potential GDP
    "CPI":    "CPIAUCSL",   # CPI all items
    "IP":     "INDPRO",     # Industrial production index
    "FFR":    "FEDFUNDS"    # Effective fed funds rate
}

# 1.5 — Download into a single DataFrame
data = pd.concat(
    {name: web.DataReader(code, "fred", start, end)
     for name, code in tickers.items()},
    axis=1
)

# 1.6 — Quick previewssssssss
data.tail()
